# Real stuff

In [1]:
from comet_ml import Experiment, ExistingExperiment
import scanpy as sc
import scirpy as ir
import pandas as pd
import torch
import yaml
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../')
import tcr_embedding as tcr

In [3]:
%load_ext autoreload
%autoreload 2

# Load dataset

### 10x Dataset

In [4]:
adata = sc.read_h5ad('../data/10x_CD8TC/highly_var_5000.h5ad')
adata

AnnData object with n_obs × n_vars = 178356 × 19707
    obs: 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_cdr3', 'IR_VJ_2_cdr3', 'IR_VDJ_1_cdr3', 'IR_VDJ_2_cdr3', 'IR_VJ_1_cdr3_nt', 'IR_VJ_2_cdr3_nt', 'IR_VDJ_1_cdr3_nt', 'IR_VDJ_2_cdr3_nt', 'IR_VJ_1_expr', 'IR_VJ_2_expr', 'IR_VDJ_1_expr', 'IR_VDJ_2_expr', 'IR_VJ_1_expr_raw', 'IR_VJ_2_expr_raw', 'IR_VDJ_1_expr_raw', 'IR_VDJ_2_expr_raw', 'IR_VJ_1_v_gene', 'IR_VJ_2_v_gene', 'IR_VDJ_1_v_gene', 'IR_VDJ_2_v_gene', 'IR_VJ_1_d_gene', 'IR_VJ_2_d_gene', 'IR_VDJ_1_d_gene', 'IR_VDJ_2_d_gene', 'IR_VJ_1_j_gene', 'IR_VJ_2_j_gene', 'IR_VDJ_1_j_gene', 'IR_VDJ_2_j_gene', 'IR_VJ_1_c_gene', 'IR_VJ_2_c_gene', 'IR_VDJ_1_c_gene', 'IR_VDJ_2_c_gene', 'IR_VJ_1_junction_ins', 'IR_VJ_2_junction_ins', 'IR_VDJ_1_junction_ins', 'IR_VDJ_2_junction_ins', 'has_ir', 'multi_chain', 'barcode', 'donor', 'cell_clono_cdr3_aa', 'cell_clono_cdr3_nt', 'CD3', 'CD19', 'CD45RA', 'CD4', 'CD8a', 'CD14', 'CD45RO', 'CD279_PD-1', 'IgG1', 'IgG2a', 'IgG2

In [5]:
adata = adata[:, adata.var['highly_variable']]
adata.shape

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


(178356, 5000)

In [6]:
(adata.X < 0)

<178356x5000 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in Compressed Sparse Row format>

In [7]:
pd.set_option('display.max_columns', None)
adata.obs

,IR_VJ_1_locus,IR_VJ_2_locus,IR_VDJ_1_locus,IR_VDJ_2_locus,IR_VJ_1_cdr3,IR_VJ_2_cdr3,IR_VDJ_1_cdr3,IR_VDJ_2_cdr3,IR_VJ_1_cdr3_nt,IR_VJ_2_cdr3_nt,IR_VDJ_1_cdr3_nt,IR_VDJ_2_cdr3_nt,IR_VJ_1_expr,IR_VJ_2_expr,IR_VDJ_1_expr,IR_VDJ_2_expr,IR_VJ_1_expr_raw,IR_VJ_2_expr_raw,IR_VDJ_1_expr_raw,IR_VDJ_2_expr_raw,IR_VJ_1_v_gene,IR_VJ_2_v_gene,IR_VDJ_1_v_gene,IR_VDJ_2_v_gene,IR_VJ_1_d_gene,IR_VJ_2_d_gene,IR_VDJ_1_d_gene,IR_VDJ_2_d_gene,IR_VJ_1_j_gene,IR_VJ_2_j_gene,IR_VDJ_1_j_gene,IR_VDJ_2_j_gene,IR_VJ_1_c_gene,IR_VJ_2_c_gene,IR_VDJ_1_c_gene,IR_VDJ_2_c_gene,IR_VJ_1_junction_ins,IR_VJ_2_junction_ins,IR_VDJ_1_junction_ins,IR_VDJ_2_junction_ins,has_ir,multi_chain,barcode,donor,cell_clono_cdr3_aa,cell_clono_cdr3_nt,CD3,CD19,CD45RA,CD4,CD8a,CD14,CD45RO,CD279_PD-1,IgG1,IgG2a,IgG2b,CD127,CD197_CCR7,HLA-DR,A0101_VTEHDTLLY_IE-1_CMV,A0201_KTWGQYWQV_gp100_Cancer,A0201_ELAGIGILTV_MART-1_Cancer,A0201_CLLWSFQTSA_Tyrosinase_Cancer,A0201_IMDQVPFSV_gp100_Cancer,A0201_SLLMWITQV_NY-ESO-1_Cancer,A0201_KVAELVHFL_MAGE-A3_Cancer,A0201_KVLEYVIKV_MAGE-A1_Cancer,A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase,A0201_LLDFVRFMGV_EBNA-3B_EBV,A0201_LLMGTLGIVC_HPV-16E7_82-91,A0201_CLGGLLTMV_LMP-2A_EBV,A0201_YLLEMLWRL_LMP1_EBV,A0201_FLYALALLL_LMP2A_EBV,A0201_GILGFVFTL_Flu-MP_Influenza,A0201_GLCTLVAML_BMLF1_EBV,A0201_NLVPMVATV_pp65_CMV,A0201_ILKEPVHGV_RT_HIV,A0201_FLASKIGRLV_Ca2-indepen-Plip-A2,A2402_CYTWNQMNL_WT1-(235-243)236M_Y,A0201_RTLNAWVKV_Gag-protein_HIV,A0201_KLQCVDLHV_PSA146-154,A0201_LLFGYPVYV_HTLV-1,A0201_SLFNTVATL_Gag-protein_HIV,A0201_SLYNTVATLY_Gag-protein_HIV,A0201_SLFNTVATLY_Gag-protein_HIV,A0201_RMFPNAPYL_WT-1,A0201_YLNDHLEPWI_BCL-X_Cancer,A0201_MLDLQPETT_16E7_HPV,A0301_KLGGALQAK_IE-1_CMV,A0301_RLRAEAQVK_EMNA-3A_EBV,A0301_RIAAWMATY_BCL-2L1_Cancer,A1101_IVTDFSVIK_EBNA-3B_EBV,A1101_AVFDRKSDAK_EBNA-3B_EBV,B3501_IPSINVHHY_pp65_CMV,A2402_AYAQKIFKI_IE-1_CMV,A2402_QYDPVAALF_pp65_CMV,B0702_QPRAPIRPI_EBNA-6_EBV,B0702_TPRVTGGGAM_pp65_CMV,B0702_RPPIFIRRL_EBNA-3A_EBV,B0702_RPHERNGFTVL_pp65_CMV,B0801_RAKFKQLL_BZLF1_EBV,B0801_ELRRKMMYM_IE-1_CMV,B0801_FLRGRAYGL_EBNA-3A_EBV,A0101_SLEGGGLGY_NC,A0101_STEGGGLAY_NC,A0201_ALIAPVHAV_NC,A2402_AYSSAGASI_NC,B0702_GPAESAAGL_NC,NR(B0801)_AAKGRGAAL_NC,A0101_VTEHDTLLY_IE-1_CMV_binder,A0201_KTWGQYWQV_gp100_Cancer_binder,A0201_ELAGIGILTV_MART-1_Cancer_binder,A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder,A0201_IMDQVPFSV_gp100_Cancer_binder,A0201_SLLMWITQV_NY-ESO-1_Cancer_binder,A0201_KVAELVHFL_MAGE-A3_Cancer_binder,A0201_KVLEYVIKV_MAGE-A1_Cancer_binder,A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder,A0201_LLDFVRFMGV_EBNA-3B_EBV_binder,A0201_LLMGTLGIVC_HPV-16E7_82-91_binder,A0201_CLGGLLTMV_LMP-2A_EBV_binder,A0201_YLLEMLWRL_LMP1_EBV_binder,A0201_FLYALALLL_LMP2A_EBV_binder,A0201_GILGFVFTL_Flu-MP_Influenza_binder,A0201_GLCTLVAML_BMLF1_EBV_binder,A0201_NLVPMVATV_pp65_CMV_binder,A0201_ILKEPVHGV_RT_HIV_binder,A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder,A2402_CYTWNQMNL_WT1-(235-243)236M_Y_binder,A0201_RTLNAWVKV_Gag-protein_HIV_binder,A0201_KLQCVDLHV_PSA146-154_binder,A0201_LLFGYPVYV_HTLV-1_binder,A0201_SLFNTVATL_Gag-protein_HIV_binder,A0201_SLYNTVATLY_Gag-protein_HIV_binder,A0201_SLFNTVATLY_Gag-protein_HIV_binder,A0201_RMFPNAPYL_WT-1_binder,A0201_YLNDHLEPWI_BCL-X_Cancer_binder,A0201_MLDLQPETT_16E7_HPV_binder,A0301_KLGGALQAK_IE-1_CMV_binder,A0301_RLRAEAQVK_EMNA-3A_EBV_binder,A0301_RIAAWMATY_BCL-2L1_Cancer_binder,A1101_IVTDFSVIK_EBNA-3B_EBV_binder,A1101_AVFDRKSDAK_EBNA-3B_EBV_binder,B3501_IPSINVHHY_pp65_CMV_binder,A2402_AYAQKIFKI_IE-1_CMV_binder,A2402_QYDPVAALF_pp65_CMV_binder,B0702_QPRAPIRPI_EBNA-6_EBV_binder,B0702_TPRVTGGGAM_pp65_CMV_binder,B0702_RPPIFIRRL_EBNA-3A_EBV_binder,B0702_RPHERNGFTVL_pp65_CMV_binder,B0801_RAKFKQLL_BZLF1_EBV_binder,B0801_ELRRKMMYM_IE-1_CMV_binder,B0801_FLRGRAYGL_EBNA-3A_EBV_binder,A0101_SLEGGGLGY_NC_binder,A0101_STEGGGLAY_NC_binder,A0201_ALIAPVHAV_NC_binder,A2402_AYSSAGASI_NC_binder,B0702_GPAESAAGL_NC_binder,NR(B0801)_AAKGRGAAL_NC_binder,n_counts,log_counts,n_genes,mt_fraction,doublet_score,doublet,louvain
AAACGGGAGAAGATTC-1-donor_

## Preprocessing

### Clean multichain, and orphaned cells

In [8]:
print('Number of multichains: ', (adata.obs['multi_chain'] == 'True').sum())

Number of multichains:  147


In [9]:
adata[adata.obs['multi_chain'] == 'False']
adata.shape

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


(178356, 5000)

In [10]:
print('Number of NaNs in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'None').sum())

print('Number of NaNs in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'None').sum())

Number of NaNs in TRA:  25491
Number of None in TRA:  20098
Number of NaNs in TRB:  25491
Number of None in TRB:  4752


In [11]:
adata = adata[(adata.obs['IR_VJ_1_cdr3'] != 'None') & (adata.obs['IR_VJ_1_cdr3'] != 'nan') & 
              (adata.obs['IR_VDJ_1_cdr3'] != 'None') & (adata.obs['IR_VDJ_1_cdr3'] != 'nan')]
adata.shape

(128731, 5000)

In [12]:
print('Number of NaNs in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'None').sum())

print('Number of NaNs in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'None').sum())

Number of NaNs in TRA:  0
Number of None in TRA:  0
Number of NaNs in TRB:  0
Number of None in TRB:  0


### Define Clonotypes

#### Check number of clonotypes with primitive count methods

Concatenate TRA and TRB sequence with '+' symbol in between

In [33]:
adata.obs['TRA+TRB'] = adata.obs['IR_VJ_1_cdr3'].astype(str) + '+' + adata.obs['IR_VDJ_1_cdr3'].astype(str)
adata.obs['TRA+TRB']

AAACGGGAGAAGATTC-1-donor_1       CAMREGSNDMRF+CASSIKGPFRTEAFF
AAACGGGTCGGACAAG-1-donor_1     CAREHMDSNYQLIW+CASSQLGRGDNEQFF
AAAGATGGTACAGACG-1-donor_1     CAALYNFNKFYF+CSARGLTADKDNYEQYF
AAAGTAGAGACGCTTT-1-donor_1    CAVIVWGNNRLAF+CASSLDRRLAGQETQYF
AAAGTAGAGCGCTTAT-1-donor_1       CAGDPDAGNNRKLIW+CAISEATDTQYF
                                           ...               
TTTGTCAGTACCCAAT-8-donor_4        CIVDTGANSKLTF+CASSESLDEKLFF
TTTGTCAGTCTAAACC-8-donor_4    CAMRGLEGGGNKLTF+CASSQDRGFGETQYF
TTTGTCAGTCTCTCGT-8-donor_4    CILRDGKGGGSQGNLIF+CASSSREPRELFF
TTTGTCATCCCACTTG-8-donor_4       CAVRDLMDTGRRALTF+CSATGQETQYF
TTTGTCATCTCTAAGG-8-donor_4     CGAIPTQGGSEKLVF+CATSGTENTGELFF
Name: TRA+TRB, Length: 128731, dtype: object

In [34]:
print(adata.obs['TRA+TRB'].value_counts()[adata.obs['TRA+TRB'].value_counts() > 10].sum())
adata.obs['TRA+TRB'].value_counts()[adata.obs['TRA+TRB'].value_counts() > 10].head(60)

68908


CAASETSYDKVIF+CASSFSGNTGELFF           5333
CAVDLMKTSYDKVIF+CASRTGLASTDTQYF        3837
CADSGGGADGLTF+CASSLRDGSEAFF            3826
CAIGPGNMLTF+CSVDLEANYGYTF              2980
CAGHTGNQFYF+CASSWGGGSHYGYTF            2210
CAAKEGYSTLTF+CASSSTGGGEKDQPQHF         2126
CAVNWEEQGGSYIPTF+CASRVSGSLSYNEQFF      1131
CAGGGSQGNLIF+CASSIRSSYEQYF             1080
CVGRKGSARQLTF+CSARDLEVLSYEQYF           929
CAMKSTNAGKSTF+CASSLGVAGTNTGELFF         924
CAVGDNFNKFYF+CASSLYSATGELFF             872
CAFIGDAGGTSYGKLTF+CASSQDHRMGGHEKLFF     839
CAEMEGGSYIPTF+CASSQRPSEVGELFF           830
CAVEPLYGNKLVF+CATSDRLAGGELFF            825
CARGGGFKTIF+CASSPVTGGGSGANVLTF          693
CGTEYPNDYKLSF+CASSSGLVSNTGELFF          671
CAVRDGGMGSNTGKLIF+CASSPMDTQYF           654
CALRAGNQFYF+CASSLMRGGTYNSPLHF           647
CAVIQINDYKLSF+CASSLGQDGYTF              645
CILWVAAGNKLTF+CASSVRSSMNTEAFF           636
CAARVRGFGNVLHC+CASSLYSATGELFF           533
CGAAGNMLTF+CASGMTGLTSEQYF               525
CAETHSGGSNYKLTF+CSVGSGDHGEQFF   

In [35]:
print(adata.obs['IR_VJ_1_cdr3'].value_counts()[adata.obs['IR_VJ_1_cdr3'].value_counts() > 10].sum())
adata.obs['IR_VJ_1_cdr3'].value_counts()[adata.obs['IR_VJ_1_cdr3'].value_counts() > 10].head(60)

75132


CAASETSYDKVIF          6220
CAVDLMKTSYDKVIF        3981
CADSGGGADGLTF          3924
CAIGPGNMLTF            3083
CAGHTGNQFYF            2253
CAAKEGYSTLTF           2216
CAGGGSQGNLIF           1629
CAVNWEEQGGSYIPTF       1187
CVGRKGSARQLTF           945
CAMKSTNAGKSTF           936
CAVGDNFNKFYF            930
CAFIGDAGGTSYGKLTF       870
CAEMEGGSYIPTF           854
CAVEPLYGNKLVF           842
CARGGGFKTIF             702
CGTEYPNDYKLSF           685
CAVRDGGMGSNTGKLIF       682
CAVIQINDYKLSF           674
CALRAGNQFYF             656
CILWVAAGNKLTF           649
CAARVRGFGNVLHC          557
CGAAGNMLTF              542
CAGAGSQGNLIF            496
CAVMDSNYQLIW            494
CAETHSGGSNYKLTF         493
CAVRDSNYQLIW            463
CAVSAASGGSYIPTF         450
CAMRRPISSGSARQLTF       427
CAMRGPISGGYNKLIF        419
CAFLGGANNLFF            418
CAVAKDLQNSGGGADGLTF     412
CIVRGADQGAQKLVF         408
CVVTETSYDKVIF           390
CVGYTNAGKSTF            386
CAVTRDDKIIF             386
CAMNPAWGGATNKLIF    

In [36]:
print(adata.obs['IR_VDJ_1_cdr3'].value_counts()[adata.obs['IR_VDJ_1_cdr3'].value_counts() > 10].sum())
adata.obs['IR_VDJ_1_cdr3'].value_counts()[adata.obs['IR_VDJ_1_cdr3'].value_counts() > 10].head(60)

72986


CASSFSGNTGELFF        5474
CASSLRDGSEAFF         3910
CASRTGLASTDTQYF       3901
CSVDLEANYGYTF         3029
CASSWGGGSHYGYTF       2305
CASSIRSSYEQYF         2261
CASSSTGGGEKDQPQHF     2198
CASSLYSATGELFF        1424
CASRVSGSLSYNEQFF      1146
CASSLGVAGTNTGELFF      966
CSARDLEVLSYEQYF        954
CASSQDHRMGGHEKLFF      879
CATSDRLAGGELFF         868
CASSQRPSEVGELFF        849
CASSPVTGGGSGANVLTF     720
CASSSGLVSNTGELFF       683
CASSPMDTQYF            675
CASSVRSSMNTEAFF        671
CASSLMRGGTYNSPLHF      666
CASSLGQDGYTF           659
CASSGPGGVDEQYF         592
CASSTDRAAQPQHF         577
CASGMTGLTSEQYF         565
CSVGSGDHGEQFF          494
CASSPRDRERGEQYF        457
CASSPAGGTDTQYF         453
CASKDGISYEQYF          439
CSVSASGGDEQYF          437
CASSQGRDRHGYTF         421
CASSLEWGGETQYF         418
CASSARSSTEQYF          411
CASSPHFQVDTGELFF       407
CASSSRDGGTDTQYF        402
CASSFLDRGPNTEAFF       395
CSASPGDYEQYF           384
CASSQEWLAVSTDTQYF      357
CASSTSTGTGYGYTF        348
C

In [65]:
clono_dict = []
for donor_id in range(1,5):
    clono_dict.append(pd.read_csv(f'../data/10x_CD8TC/patient_{donor_id}/vdj_v1_hs_aggregated_donor{donor_id}_clonotypes.csv'))
clono_dict = pd.concat(clono_dict, ignore_index=True)
clono_dict = {aa: clono_id for clono_id, aa in enumerate(clono_dict['cdr3s_aa'].unique())}
clono_dict

{'TRA:CAGHTGNQFYF;TRB:CASSWGGGSHYGYTF': 0,
 'TRA:CAARVRGFGNVLHC;TRA:CAVGDNFNKFYF;TRB:CASSLYSATGELFF': 1,
 'TRB:CASSWGGGSHYGYTF': 2,
 'TRA:CAVSAASGGSYIPTF;TRB:CASSPRDRERGEQYF': 3,
 'TRA:CAMNPAWGGATNKLIF;TRB:CSASPGDYEQYF': 4,
 'TRA:CAVHGYGQNFVF;TRB:CASSQEWLAVSTDTQYF': 5,
 'TRA:CAASGYDYKLSF;TRA:CAVFLYGNNRLAF;TRB:CSVSASGGDEQYF': 6,
 'TRA:CAREHMDSNYQLIW;TRB:CASSQLGRGDNEQFF': 7,
 'TRA:CAEGGNTGKLIF;TRB:CASSPGGLAGADTQYF': 8,
 'TRA:CALKLIKAAGNKLTF;TRB:CAWTGTGKIGWDSPLHF': 9,
 'TRA:CAVDFRDNYGQNFVF;TRB:CASSLSSGTYEQYF': 10,
 'TRA:CAVGDNFNKFYF;TRB:CASSLYSATGELFF': 11,
 'TRA:CAVDPMDTGRRALTF;TRB:CASSLGFTDTQYF': 12,
 'TRA:CATEGASGGSYIPTF;TRB:CAWVLGPAGDTQYF': 13,
 'TRA:CAVRSSGGSYIPTF;TRB:CATSTGDSNQPQHF': 14,
 'TRA:CAESKNNNARLMF;TRB:CASSLSGYEQYF': 15,
 'TRA:CAARVRGFGNVLHC;TRB:CASSLYSATGELFF': 16,
 'TRA:CAASSAGAGSYQLTF;TRB:CASSLFGGTYEQYF': 17,
 'TRA:CAASVSIWTGTASKLTF;TRA:CAAWDMEYGNKLVF;TRB:CAISDPGLAGGGGEQFF': 18,
 'TRA:CAVTDGGSQGNLIF;TRB:CASSIRSSYEQYF': 19,
 'TRA:CALGDSGAGSYQLTF;TRB:CASREIQGSGANVLTF': 20,

In [74]:
len(clono_dict)

83803

In [77]:
print('Number of cells without assigned clonotype', adata.obs['cell_clono_cdr3_aa'].map(clono_dict).isna().sum())
adata.obs['clonotype'] = adata.obs['cell_clono_cdr3_aa'].map(clono_dict)

Number of cells without assigned clonotype 0


In [78]:
adata.obs['clonotype'].value_counts()

577      5294
67605    3794
622      3791
49577    2930
0        2187
         ... 
54644       1
54784       1
64685       1
51249       1
33079       1
Name: clonotype, Length: 52619, dtype: int64

### Represent aa chains as label sequence or one-hot encoding

sanity check if data only contains the correct aa tokens

In [14]:
aa_tokens = adata.obs['TRA+TRB'].apply(lambda x: list(x))
unique_aa_tokens = sorted(set([x for sublist in aa_tokens for x in sublist]))
unique_aa_tokens

['+',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y']

Get maximum sequence length as pad value

In [15]:
pad = adata.obs['TRA+TRB'].str.len().max()
pad = int(pad)
pad

45

encode amino acids to unique id, '_' is pad character, '<' start and '>' end symbol '+' is the separation between TRA and TRB

In [16]:
aa_to_id = {'_': 0, 'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13,
            'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20, '+': 21, '<': 22, '>': 23}

In [17]:
# Returns aa_to_id, but we don't need it here. Only needed if the method should determine aa_to_id by itself
_ = tcr.utils.aa_encoding(adata, read_col='TRA+TRB', ohe_col='one_hot', label_col='tcr_seq', length_col='seq_len', pad=pad, aa_to_id=aa_to_id, start_end_symbol=True)
adata.obs[['TRA+TRB', 'tcr_seq', 'one_hot', 'seq_len']]

,TRA+TRB,tcr_seq,one_hot,seq_len
AAACGGGAGAAGATTC-1-donor_1,<CAMREGSNDMRF+CASSIKGPFRTEAFF>,"[22, 2, 1, 11, 15, 4, 6, 16, 12, 3, 11, 15, 5,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",30
AAACGGGTCGGACAAG-1-donor_1,<CAREHMDSNYQLIW+CASSQLGRGDNEQFF>,"[22, 2, 1, 15, 4, 7, 11, 3, 16, 12, 20, 14, 10...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",32
AAAGATGGTACAGACG-1-donor_1,<CAALYNFNKFYF+CSARGLTADKDNYEQYF>,"[22, 2, 1, 1, 10, 20, 12, 5, 12, 9, 5, 20, 5, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",32
AAAGTAGAGACGCTTT-1-donor_1,<CAVIVWGNNRLAF+CASSLDRRLAGQETQYF>,"[22, 2, 1, 18, 8, 18, 19, 6, 12, 12, 15, 10, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",33
AAAGTAGAGCGCTTAT-1-donor_1,<CAGDPDAGNNRKLIW+CAISEATDTQYF>,"[22, 2, 1, 6, 3, 13, 3, 1, 6, 12, 12, 15, 9, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",30
...,...,...,...,...
TTTGTCAGTACCCAAT-8-donor_4,<CIVDTGANSKLTF+CASSESLDEKLFF>,"[22, 2, 8, 18, 3, 17, 6, 1, 12, 16, 9, 10, 17,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",29
TTTGTCAGTCTAAACC-8-donor_4,<CAMRGLEGGGNKLTF+CASSQDRGFGETQYF>,"[22, 2, 1, 11, 15, 6, 10, 4, 6, 6, 6, 12, 9, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",33
TTTGTCAGTCTCTCGT-8-donor_4,<CILRDGKGGGSQGNLIF+CASSSREPRELFF>,"[22, 2, 8, 10, 15, 3, 6, 9, 6, 6, 6, 16, 14, 6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",33
TTTGTCATCCCACTTG-8-donor_4,<CAVRDLMDTGRRALTF+CSATGQETQYF>,"[22, 2, 1, 18, 15, 3, 10, 11, 3, 17, 6, 15, 15...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",30


### Antigen Binding Preprocessing

In [18]:
# antigen binding list
antigen_binding_list = ['A0101_VTEHDTLLY_IE-1_CMV_binder', 
                        'A0201_KTWGQYWQV_gp100_Cancer_binder', 
                        'A0201_ELAGIGILTV_MART-1_Cancer_binder', 
                        'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder', 
                        'A0201_IMDQVPFSV_gp100_Cancer_binder', 
                        'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder', 
                        'A0201_KVAELVHFL_MAGE-A3_Cancer_binder', 
                        'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder', 
                        'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder', 
                        'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder', 
                        'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder', 
                        'A0201_CLGGLLTMV_LMP-2A_EBV_binder', 
                        'A0201_YLLEMLWRL_LMP1_EBV_binder', 
                        'A0201_FLYALALLL_LMP2A_EBV_binder', 
                        'A0201_GILGFVFTL_Flu-MP_Influenza_binder', 
                        'A0201_GLCTLVAML_BMLF1_EBV_binder', 
                        'A0201_NLVPMVATV_pp65_CMV_binder', 
                        'A0201_ILKEPVHGV_RT_HIV_binder', 
                        'A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder', 
                        'A2402_CYTWNQMNL_WT1-(235-243)236M_Y_binder', 
                        'A0201_RTLNAWVKV_Gag-protein_HIV_binder', 
                        'A0201_KLQCVDLHV_PSA146-154_binder', 
                        'A0201_LLFGYPVYV_HTLV-1_binder', 
                        'A0201_SLFNTVATL_Gag-protein_HIV_binder', 
                        'A0201_SLYNTVATLY_Gag-protein_HIV_binder', 
                        'A0201_SLFNTVATLY_Gag-protein_HIV_binder', 
                        'A0201_RMFPNAPYL_WT-1_binder', 
                        'A0201_YLNDHLEPWI_BCL-X_Cancer_binder', 
                        'A0201_MLDLQPETT_16E7_HPV_binder', 
                        'A0301_KLGGALQAK_IE-1_CMV_binder', 
                        'A0301_RLRAEAQVK_EMNA-3A_EBV_binder', 
                        'A0301_RIAAWMATY_BCL-2L1_Cancer_binder',
                        'A1101_IVTDFSVIK_EBNA-3B_EBV_binder', 
                        'A1101_AVFDRKSDAK_EBNA-3B_EBV_binder', 
                        'B3501_IPSINVHHY_pp65_CMV_binder', 
                        'A2402_AYAQKIFKI_IE-1_CMV_binder',
                        'A2402_QYDPVAALF_pp65_CMV_binder',
                        'B0702_QPRAPIRPI_EBNA-6_EBV_binder',
                        'B0702_TPRVTGGGAM_pp65_CMV_binder',
                        'B0702_RPPIFIRRL_EBNA-3A_EBV_binder',
                        'B0702_RPHERNGFTVL_pp65_CMV_binder',
                        'B0801_RAKFKQLL_BZLF1_EBV_binder', 
                        'B0801_ELRRKMMYM_IE-1_CMV_binder', 
                        'B0801_FLRGRAYGL_EBNA-3A_EBV_binder',
                        'A0101_SLEGGGLGY_NC_binder', 
                        'A0101_STEGGGLAY_NC_binder',
                        'A0201_ALIAPVHAV_NC_binder', 
                        'A2402_AYSSAGASI_NC_binder',
                        'B0702_GPAESAAGL_NC_binder',
                        'NR(B0801)_AAKGRGAAL_NC_binder']

Create matrix, that determines antigen binding of each cell

In [19]:
binding_matrix = (adata.obs[antigen_binding_list] == 'True').values.astype(int)
binding_matrix.shape

(128731, 50)

Check if binding is limited to only one antigen
Result: Yes, every cell can only bind to one specific antigen, or binding is not determined

In [20]:
binding_matrix.sum(axis=1).shape  # just check the dimensions are correct and we have number of bindings for each cell

(128731,)

In [21]:
unique_value, count = np.unique(binding_matrix.sum(axis=1), return_counts=True)
for unique_value_, count_ in zip(unique_value, count):
    print(f'Cells with {unique_value_} bindings: {count_}')

Cells with 0 bindings: 66874
Cells with 1 bindings: 61857


Save antigen binding to column

In [22]:
label_to_binding = {label_id: antigen for label_id, antigen in enumerate(antigen_binding_list)}
label_to_binding[-1] = 'no_data'
binding_to_label = {antigen: label_id for label_id, antigen in enumerate(antigen_binding_list)}
binding_to_label['no_data'] = -1
binding_to_label == {v: k for k, v in label_to_binding.items()}  # sanity check, if both dicts are the reverse of each other

True

In [23]:
adata.obs['has_binding'] = binding_matrix.sum(axis=1).astype(bool)
adata.obs['has_binding']

AAACGGGAGAAGATTC-1-donor_1     True
AAACGGGTCGGACAAG-1-donor_1    False
AAAGATGGTACAGACG-1-donor_1    False
AAAGTAGAGACGCTTT-1-donor_1    False
AAAGTAGAGCGCTTAT-1-donor_1    False
                              ...  
TTTGTCAGTACCCAAT-8-donor_4    False
TTTGTCAGTCTAAACC-8-donor_4    False
TTTGTCAGTCTCTCGT-8-donor_4    False
TTTGTCATCCCACTTG-8-donor_4    False
TTTGTCATCTCTAAGG-8-donor_4    False
Name: has_binding, Length: 128731, dtype: bool

In [24]:
adata.obs['binding_label'] = np.argmax(binding_matrix, axis=1)
adata.obs['binding_label'][~adata.obs['has_binding']] = -1
adata.obs['binding_label'].head(15)

<ipython-input-24-6fb9f26126ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['binding_label'][~adata.obs['has_binding']] = -1


AAACGGGAGAAGATTC-1-donor_1    29
AAACGGGTCGGACAAG-1-donor_1    -1
AAAGATGGTACAGACG-1-donor_1    -1
AAAGTAGAGACGCTTT-1-donor_1    -1
AAAGTAGAGCGCTTAT-1-donor_1    -1
AAAGTAGAGTTACGGG-1-donor_1    -1
AACCGCGGTCTGCGGT-1-donor_1    -1
AACGTTGTCGGATGTT-1-donor_1    -1
AACTCAGCAGGTTTCA-1-donor_1    -1
AACTGGTGTGCGAAAC-1-donor_1    -1
AACTTTCAGTAACCCT-1-donor_1    32
AACTTTCCATTACCTT-1-donor_1    -1
AAGGTTCCATTAGGCT-1-donor_1    -1
ACACCAACAGTCGATT-1-donor_1    -1
ACACCCTAGGAGTTTA-1-donor_1    29
Name: binding_label, dtype: int64

In [25]:
adata.obs['binding_name'] = adata.obs['binding_label'].map(label_to_binding)
adata.obs['binding_name'].head(15)

AAACGGGAGAAGATTC-1-donor_1       A0301_KLGGALQAK_IE-1_CMV_binder
AAACGGGTCGGACAAG-1-donor_1                               no_data
AAAGATGGTACAGACG-1-donor_1                               no_data
AAAGTAGAGACGCTTT-1-donor_1                               no_data
AAAGTAGAGCGCTTAT-1-donor_1                               no_data
AAAGTAGAGTTACGGG-1-donor_1                               no_data
AACCGCGGTCTGCGGT-1-donor_1                               no_data
AACGTTGTCGGATGTT-1-donor_1                               no_data
AACTCAGCAGGTTTCA-1-donor_1                               no_data
AACTGGTGTGCGAAAC-1-donor_1                               no_data
AACTTTCAGTAACCCT-1-donor_1    A1101_IVTDFSVIK_EBNA-3B_EBV_binder
AACTTTCCATTACCTT-1-donor_1                               no_data
AAGGTTCCATTAGGCT-1-donor_1                               no_data
ACACCAACAGTCGATT-1-donor_1                               no_data
ACACCCTAGGAGTTTA-1-donor_1       A0301_KLGGALQAK_IE-1_CMV_binder
Name: binding_name, dtype

In [26]:
# (adata.obs['has_binding'].astype(bool) == (adata.obs[antigen_binding_list] == 'True').sum(1).astype(bool)).all()

# Initialize and train model

In [27]:
config_name = 'bigru_paper_oriented'
with open(f'../config/{config_name}.yaml') as file:
     params = yaml.load(file)
experiment_name = '10x_random_split' + config_name
params

<ipython-input-27-56a9b31d0c69>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  params = yaml.load(file)


{'seq_model_arch': 'BiGRU',
 'seq_model_hyperparams': {'embedding_dim': 64,
  'hidden_size': 256,
  'num_layers': 2,
  'dropout': 0.1,
  'bidirectional': True},
 'scRNA_model_arch': 'MLP',
 'scRNA_model_hyperparams': {'gene_hidden': [800],
  'activation': 'leakyrelu',
  'output_activation': 'relu',
  'dropout': 0.2,
  'batch_norm': True},
 'hdim': 800,
 'activation': 'leakyrelu',
 'dropout': 0.2,
 'batch_norm': True,
 'shared_hidden': [200],
 'zdim': 100,
 'lr': 0.001,
 'batch_size': 256,
 'losses': ['MSE', 'CE'],
 'loss_weights': [0.1, 1.0, 5e-05]}

In [28]:
with open('../comet_ml_key/API_key.txt') as f:
    COMET_ML_KEY = f.read()

In [29]:
# experiment = ExistingExperiment(api_key=COMET_ML_KEY, previous_experiment='e64f46cf7b1a44fcaa87b1dac29baf0d')
experiment = Experiment(api_key=COMET_ML_KEY, workspace='tcr', project_name='10x_GRU')

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/tcr/10x-gru/e64f46cf7b1a44fcaa87b1dac29baf0d



In [30]:
experiment.log_parameters(params)
experiment.log_parameter('experiment_name', experiment_name)

In [31]:
train_mask = np.zeros(len(adata))
train_mask[:int(len(adata)*0.8)] = 1
np.random.shuffle(train_mask)
train_mask = train_mask.astype(bool)
torch.save(train_mask, f'../saved_models/{experiment_name}_train_test_mask')
# train_mask = torch.load(f'../saved_models/{experiment_name}_train_test_mask')
train_mask

array([ True,  True,  True, ..., False,  True,  True])

In [32]:
train_adata = adata[train_mask]
train_adata = train_adata[train_adata.obs['has_binding']]
test_adata = adata[~train_mask]
test_adata = test_adata[test_adata.obs['has_binding']]

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [33]:
model = tcr.models.joint_model.JointModel(
    adatas=[train_adata],  # adatas containing gene expression and TCR-seq
    names=['10x'],
    aa_to_id = aa_to_id,  # dict {aa_char: id}
    seq_model_arch=params['seq_model_arch'],  # seq model architecture
    seq_model_hyperparams=params['seq_model_hyperparams'],  # dict of seq model hyperparameters
    scRNA_model_arch=params['scRNA_model_arch'],
    scRNA_model_hyperparams=params['scRNA_model_hyperparams'],
    zdim=params['zdim'],  # zdim
    hdim=params['hdim'],  # hidden dimension of scRNA and seq encoders
    activation=params['activation'],  # activation function of autoencoder hidden layers
    dropout=params['dropout'],
    batch_norm=params['batch_norm'],
    shared_hidden=params['shared_hidden'],  # hidden layers of shared encoder / decoder
    gene_layers=[],  # [] or list of str for layer keys of each dataset
    seq_keys=[]  # [] or list of str for seq keys of each dataset
)

In [34]:
# print model architecture
model.model

JointModelTorch(
  (seq_encoder): BiGRUEncoder(
    (embedding): Embedding(24, 64, padding_idx=0)
    (gru): GRU(64, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (output_layer): Linear(in_features=1024, out_features=800, bias=True)
  )
  (seq_decoder): BiGRUDecoder(
    (embedding): Embedding(24, 64, padding_idx=0)
    (hidden_state_layer): Linear(in_features=1600, out_features=512, bias=True)
    (gru): GRU(64, 256, num_layers=2, batch_first=True, dropout=0.1)
    (output_layer): Linear(in_features=256, out_features=24, bias=True)
  )
  (gene_encoder): MLP(
    (network): Sequential(
      (0): Sequential(
        (0): Linear(in_features=5000, out_features=800, bias=False)
        (1): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=800, out_features=800, bias=Fal

In [ ]:
# model.train(
#     experiment_name=experiment_name,
#     n_iters=None,
#     n_epochs=300,
#     batch_size=params['batch_size'],
#     lr=params['lr'],
#     losses=params['losses'],  # list of losses for each modality: losses[0] := scRNA, losses[1] := TCR
#     loss_weights=params['loss_weights'],  # [] or list of floats storing weighting of loss in order [scRNA, TCR, KLD]
#     val_split=0.25,
#     metadata=[],
#     validate_every=5,
#     print_every=5,
#     save_every=25,
#     num_workers=0,
#     verbose=1,  # 0: only tdqm progress bar, 1: val loss, 2: train and val loss
# #     continue_training=True,
#     device=None,
#     comet=experiment
# )

In [ ]:
model.history

In [ ]:
model.train_history

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10));
plt.subplot(221);
plt.plot(model.train_history['epoch'], model.train_history['loss'], '.-', label='Train loss');
plt.plot(model.history['epoch'], model.history['loss'], '.-', label='Val loss');
plt.legend();

plt.subplot(222);
plt.plot(model.train_history['epoch'], model.train_history['scRNA_loss'], '.-', label='Train scRNA loss');
plt.plot(model.history['epoch'], model.history['scRNA_loss'], '.-', label='Val scRNA loss');
plt.legend();

plt.subplot(223);
plt.plot(model.train_history['epoch'], model.train_history['TCR_loss'], '.-', label='Train TCR loss');
plt.plot(model.history['epoch'], model.history['TCR_loss'], '.-', label='Val TCR loss');
plt.xlabel('#Epochs');
plt.legend();

plt.subplot(224);
plt.plot(model.train_history['epoch'], model.train_history['KLD_loss'], '.-', label='Train KLD loss');
plt.plot(model.history['epoch'], model.history['KLD_loss'], '.-', label='Val KLD loss');
plt.xlabel('#');
plt.legend();


# UMAP Plot of latent space

### On Test Data

Filter cells with no binding data and only UMAP on high count antigen bindings

In [35]:
test_adata = test_adata[test_adata.obs['has_binding']]

In [36]:
test_adata.obs['binding_name'].value_counts()

A0301_KLGGALQAK_IE-1_CMV_binder                    5298
B0801_RAKFKQLL_BZLF1_EBV_binder                    3276
A0201_GILGFVFTL_Flu-MP_Influenza_binder            1538
A1101_IVTDFSVIK_EBNA-3B_EBV_binder                 1150
A1101_AVFDRKSDAK_EBNA-3B_EBV_binder                 677
A0201_GLCTLVAML_BMLF1_EBV_binder                    230
A0301_RLRAEAQVK_EMNA-3A_EBV_binder                   71
A0201_ELAGIGILTV_MART-1_Cancer_binder                55
A0201_FLYALALLL_LMP2A_EBV_binder                     22
A0201_LLDFVRFMGV_EBNA-3B_EBV_binder                  19
A0201_SLFNTVATLY_Gag-protein_HIV_binder               9
A0201_RTLNAWVKV_Gag-protein_HIV_binder                7
A0201_LLFGYPVYV_HTLV-1_binder                         6
A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder           6
A0201_YLLEMLWRL_LMP1_EBV_binder                       5
A2402_AYAQKIFKI_IE-1_CMV_binder                       4
B0801_FLRGRAYGL_EBNA-3A_EBV_binder                    4
A0201_KVLEYVIKV_MAGE-A1_Cancer_binder           

In [37]:
# only get antigen with high counts
antigen_count = test_adata.obs['binding_name'].value_counts()
high_antigen_count = antigen_count[antigen_count > len(test_adata)*0.01]
high_antigen_count

A0301_KLGGALQAK_IE-1_CMV_binder            5298
B0801_RAKFKQLL_BZLF1_EBV_binder            3276
A0201_GILGFVFTL_Flu-MP_Influenza_binder    1538
A1101_IVTDFSVIK_EBNA-3B_EBV_binder         1150
A1101_AVFDRKSDAK_EBNA-3B_EBV_binder         677
A0201_GLCTLVAML_BMLF1_EBV_binder            230
Name: binding_name, dtype: int64

In [38]:
test_adata

View of AnnData object with n_obs × n_vars = 12398 × 5000
    obs: 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_cdr3', 'IR_VJ_2_cdr3', 'IR_VDJ_1_cdr3', 'IR_VDJ_2_cdr3', 'IR_VJ_1_cdr3_nt', 'IR_VJ_2_cdr3_nt', 'IR_VDJ_1_cdr3_nt', 'IR_VDJ_2_cdr3_nt', 'IR_VJ_1_expr', 'IR_VJ_2_expr', 'IR_VDJ_1_expr', 'IR_VDJ_2_expr', 'IR_VJ_1_expr_raw', 'IR_VJ_2_expr_raw', 'IR_VDJ_1_expr_raw', 'IR_VDJ_2_expr_raw', 'IR_VJ_1_v_gene', 'IR_VJ_2_v_gene', 'IR_VDJ_1_v_gene', 'IR_VDJ_2_v_gene', 'IR_VJ_1_d_gene', 'IR_VJ_2_d_gene', 'IR_VDJ_1_d_gene', 'IR_VDJ_2_d_gene', 'IR_VJ_1_j_gene', 'IR_VJ_2_j_gene', 'IR_VDJ_1_j_gene', 'IR_VDJ_2_j_gene', 'IR_VJ_1_c_gene', 'IR_VJ_2_c_gene', 'IR_VDJ_1_c_gene', 'IR_VDJ_2_c_gene', 'IR_VJ_1_junction_ins', 'IR_VJ_2_junction_ins', 'IR_VDJ_1_junction_ins', 'IR_VDJ_2_junction_ins', 'has_ir', 'multi_chain', 'barcode', 'donor', 'cell_clono_cdr3_aa', 'cell_clono_cdr3_nt', 'CD3', 'CD19', 'CD45RA', 'CD4', 'CD8a', 'CD14', 'CD45RO', 'CD279_PD-1', 'IgG1', 'IgG2a',

Use last saved model

In [39]:
model.load(f'../saved_models/{experiment_name}_last_model.pt')

In [40]:
z = model.get_latent(
    adatas=[test_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Batch: 100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:03<00:00, 13.60it/s]


In [41]:
z

AnnData object with n_obs × n_vars = 12398 × 100
    obs: 'barcode', 'dataset', 'A0101_VTEHDTLLY_IE-1_CMV_binder', 'A0201_KTWGQYWQV_gp100_Cancer_binder', 'A0201_ELAGIGILTV_MART-1_Cancer_binder', 'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder', 'A0201_IMDQVPFSV_gp100_Cancer_binder', 'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder', 'A0201_KVAELVHFL_MAGE-A3_Cancer_binder', 'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder', 'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder', 'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder', 'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder', 'A0201_CLGGLLTMV_LMP-2A_EBV_binder', 'A0201_YLLEMLWRL_LMP1_EBV_binder', 'A0201_FLYALALLL_LMP2A_EBV_binder', 'A0201_GILGFVFTL_Flu-MP_Influenza_binder', 'A0201_GLCTLVAML_BMLF1_EBV_binder', 'A0201_NLVPMVATV_pp65_CMV_binder', 'A0201_ILKEPVHGV_RT_HIV_binder', 'A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder', 'A2402_CYTWNQMNL_WT1-(235-243)236M_Y_binder', 'A0201_RTLNAWVKV_Gag-protein_HIV_binder', 'A0201_KLQCVDLHV_PSA146-154_binder', 'A0201_LLFGYPVYV_HTLV-1_binder', 'A020

In [42]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [43]:
plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
    ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.3)
    experiment.log_figure(figure_name=f'test_{antigen}', figure=ax, step=model.epoch, overwrite=False)
    ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'test_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

... storing 'dataset' as categorical
... storing 'A0101_VTEHDTLLY_IE-1_CMV_binder' as categorical
... storing 'A0201_KTWGQYWQV_gp100_Cancer_binder' as categorical
... storing 'A0201_ELAGIGILTV_MART-1_Cancer_binder' as categorical
... storing 'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder' as categorical
... storing 'A0201_IMDQVPFSV_gp100_Cancer_binder' as categorical
... storing 'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder' as categorical
... storing 'A0201_KVAELVHFL_MAGE-A3_Cancer_binder' as categorical
... storing 'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder' as categorical
... storing 'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder' as categorical
... storing 'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder' as categorical
... storing 'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder' as categorical
... storing 'A0201_CLGGLLTMV_LMP-2A_EBV_binder' as categorical
... storing 'A0201_YLLEMLWRL_LMP1_EBV_binder' as categorical
... storing 'A0201_FLYALALLL_LMP2A_EBV_binder' as categorical
... storing 'A0201_GILGFVFTL_Flu-MP_

<Figure size 720x720 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

Use "best" saved model (currently based on val_loss)

In [44]:
model.load(f'../saved_models/{experiment_name}_best_model.pt')

In [45]:
z = model.get_latent(
    adatas=[test_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Batch: 100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 17.40it/s]


In [46]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [47]:
plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
    ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.2)
    experiment.log_figure(figure_name=f'test_{antigen}', figure=ax, step=model.epoch, overwrite=False)
    ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'test_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

... storing 'dataset' as categorical
... storing 'A0101_VTEHDTLLY_IE-1_CMV_binder' as categorical
... storing 'A0201_KTWGQYWQV_gp100_Cancer_binder' as categorical
... storing 'A0201_ELAGIGILTV_MART-1_Cancer_binder' as categorical
... storing 'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder' as categorical
... storing 'A0201_IMDQVPFSV_gp100_Cancer_binder' as categorical
... storing 'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder' as categorical
... storing 'A0201_KVAELVHFL_MAGE-A3_Cancer_binder' as categorical
... storing 'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder' as categorical
... storing 'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder' as categorical
... storing 'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder' as categorical
... storing 'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder' as categorical
... storing 'A0201_CLGGLLTMV_LMP-2A_EBV_binder' as categorical
... storing 'A0201_YLLEMLWRL_LMP1_EBV_binder' as categorical
... storing 'A0201_FLYALALLL_LMP2A_EBV_binder' as categorical
... storing 'A0201_GILGFVFTL_Flu-MP_

<Figure size 720x720 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### On Train Data

Filter cells with no binding data and only UMAP on high count antigen bindings

In [48]:
train_adata = train_adata[train_adata.obs['has_binding']]

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [49]:
train_adata.obs['binding_name'].value_counts()

A0301_KLGGALQAK_IE-1_CMV_binder                    21091
B0801_RAKFKQLL_BZLF1_EBV_binder                    13214
A0201_GILGFVFTL_Flu-MP_Influenza_binder             5855
A1101_IVTDFSVIK_EBNA-3B_EBV_binder                  4510
A1101_AVFDRKSDAK_EBNA-3B_EBV_binder                 2842
A0201_GLCTLVAML_BMLF1_EBV_binder                    1014
A0301_RLRAEAQVK_EMNA-3A_EBV_binder                   311
A0201_ELAGIGILTV_MART-1_Cancer_binder                186
A0201_LLDFVRFMGV_EBNA-3B_EBV_binder                  108
A0201_FLYALALLL_LMP2A_EBV_binder                     102
B0801_FLRGRAYGL_EBNA-3A_EBV_binder                    31
A0201_RTLNAWVKV_Gag-protein_HIV_binder                28
A0201_LLFGYPVYV_HTLV-1_binder                         23
A0201_SLFNTVATLY_Gag-protein_HIV_binder               19
A2402_AYAQKIFKI_IE-1_CMV_binder                       19
A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder           18
A0201_KTWGQYWQV_gp100_Cancer_binder                   12
A0201_IMDQVPFSV_gp100_Cancer_bi

In [50]:
# only get antigen with high counts
antigen_count = train_adata.obs['binding_name'].value_counts()
high_antigen_count = antigen_count[antigen_count > len(train_adata)*0.01]
high_antigen_count

A0301_KLGGALQAK_IE-1_CMV_binder            21091
B0801_RAKFKQLL_BZLF1_EBV_binder            13214
A0201_GILGFVFTL_Flu-MP_Influenza_binder     5855
A1101_IVTDFSVIK_EBNA-3B_EBV_binder          4510
A1101_AVFDRKSDAK_EBNA-3B_EBV_binder         2842
A0201_GLCTLVAML_BMLF1_EBV_binder            1014
Name: binding_name, dtype: int64

In [51]:
train_adata

View of AnnData object with n_obs × n_vars = 49459 × 5000
    obs: 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_cdr3', 'IR_VJ_2_cdr3', 'IR_VDJ_1_cdr3', 'IR_VDJ_2_cdr3', 'IR_VJ_1_cdr3_nt', 'IR_VJ_2_cdr3_nt', 'IR_VDJ_1_cdr3_nt', 'IR_VDJ_2_cdr3_nt', 'IR_VJ_1_expr', 'IR_VJ_2_expr', 'IR_VDJ_1_expr', 'IR_VDJ_2_expr', 'IR_VJ_1_expr_raw', 'IR_VJ_2_expr_raw', 'IR_VDJ_1_expr_raw', 'IR_VDJ_2_expr_raw', 'IR_VJ_1_v_gene', 'IR_VJ_2_v_gene', 'IR_VDJ_1_v_gene', 'IR_VDJ_2_v_gene', 'IR_VJ_1_d_gene', 'IR_VJ_2_d_gene', 'IR_VDJ_1_d_gene', 'IR_VDJ_2_d_gene', 'IR_VJ_1_j_gene', 'IR_VJ_2_j_gene', 'IR_VDJ_1_j_gene', 'IR_VDJ_2_j_gene', 'IR_VJ_1_c_gene', 'IR_VJ_2_c_gene', 'IR_VDJ_1_c_gene', 'IR_VDJ_2_c_gene', 'IR_VJ_1_junction_ins', 'IR_VJ_2_junction_ins', 'IR_VDJ_1_junction_ins', 'IR_VDJ_2_junction_ins', 'has_ir', 'multi_chain', 'barcode', 'donor', 'cell_clono_cdr3_aa', 'cell_clono_cdr3_nt', 'CD3', 'CD19', 'CD45RA', 'CD4', 'CD8a', 'CD14', 'CD45RO', 'CD279_PD-1', 'IgG1', 'IgG2a',

Use last saved model

In [52]:
model.load(f'../saved_models/{experiment_name}_last_model.pt')

In [53]:
z = model.get_latent(
    adatas=[train_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Batch: 100%|█████████████████████████████████████████████████████████████████████████| 194/194 [00:12<00:00, 16.12it/s]


In [54]:
z

AnnData object with n_obs × n_vars = 49459 × 100
    obs: 'barcode', 'dataset', 'A0101_VTEHDTLLY_IE-1_CMV_binder', 'A0201_KTWGQYWQV_gp100_Cancer_binder', 'A0201_ELAGIGILTV_MART-1_Cancer_binder', 'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder', 'A0201_IMDQVPFSV_gp100_Cancer_binder', 'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder', 'A0201_KVAELVHFL_MAGE-A3_Cancer_binder', 'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder', 'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder', 'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder', 'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder', 'A0201_CLGGLLTMV_LMP-2A_EBV_binder', 'A0201_YLLEMLWRL_LMP1_EBV_binder', 'A0201_FLYALALLL_LMP2A_EBV_binder', 'A0201_GILGFVFTL_Flu-MP_Influenza_binder', 'A0201_GLCTLVAML_BMLF1_EBV_binder', 'A0201_NLVPMVATV_pp65_CMV_binder', 'A0201_ILKEPVHGV_RT_HIV_binder', 'A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder', 'A2402_CYTWNQMNL_WT1-(235-243)236M_Y_binder', 'A0201_RTLNAWVKV_Gag-protein_HIV_binder', 'A0201_KLQCVDLHV_PSA146-154_binder', 'A0201_LLFGYPVYV_HTLV-1_binder', 'A020

In [55]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [56]:
# plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
    ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.3)
    experiment.log_figure(figure_name=f'train_{antigen}', figure=ax, step=model.epoch, overwrite=False)
    ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'train_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

... storing 'dataset' as categorical
... storing 'A0101_VTEHDTLLY_IE-1_CMV_binder' as categorical
... storing 'A0201_KTWGQYWQV_gp100_Cancer_binder' as categorical
... storing 'A0201_ELAGIGILTV_MART-1_Cancer_binder' as categorical
... storing 'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder' as categorical
... storing 'A0201_IMDQVPFSV_gp100_Cancer_binder' as categorical
... storing 'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder' as categorical
... storing 'A0201_KVAELVHFL_MAGE-A3_Cancer_binder' as categorical
... storing 'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder' as categorical
... storing 'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder' as categorical
... storing 'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder' as categorical
... storing 'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder' as categorical
... storing 'A0201_CLGGLLTMV_LMP-2A_EBV_binder' as categorical
... storing 'A0201_YLLEMLWRL_LMP1_EBV_binder' as categorical
... storing 'A0201_FLYALALLL_LMP2A_EBV_binder' as categorical
... storing 'A0201_GILGFVFTL_Flu-MP_

<Figure size 720x720 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

Use "best" saved model, (currently based on val loss)

In [57]:
model.load(f'../saved_models/{experiment_name}_best_model.pt')

In [58]:
z = model.get_latent(
    adatas=[train_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Batch: 100%|█████████████████████████████████████████████████████████████████████████| 194/194 [00:11<00:00, 16.45it/s]


In [59]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [60]:
# plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
    ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.2)
    experiment.log_figure(figure_name=f'train_{antigen}', figure=ax, step=model.epoch, overwrite=False)
    ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'train_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

... storing 'dataset' as categorical
... storing 'A0101_VTEHDTLLY_IE-1_CMV_binder' as categorical
... storing 'A0201_KTWGQYWQV_gp100_Cancer_binder' as categorical
... storing 'A0201_ELAGIGILTV_MART-1_Cancer_binder' as categorical
... storing 'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder' as categorical
... storing 'A0201_IMDQVPFSV_gp100_Cancer_binder' as categorical
... storing 'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder' as categorical
... storing 'A0201_KVAELVHFL_MAGE-A3_Cancer_binder' as categorical
... storing 'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder' as categorical
... storing 'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder' as categorical
... storing 'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder' as categorical
... storing 'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder' as categorical
... storing 'A0201_CLGGLLTMV_LMP-2A_EBV_binder' as categorical
... storing 'A0201_YLLEMLWRL_LMP1_EBV_binder' as categorical
... storing 'A0201_FLYALALLL_LMP2A_EBV_binder' as categorical
... storing 'A0201_GILGFVFTL_Flu-MP_

<Figure size 720x720 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

# kNN prediction

In [61]:
train_adata.obs['binding_name'].value_counts()

A0301_KLGGALQAK_IE-1_CMV_binder                    21091
B0801_RAKFKQLL_BZLF1_EBV_binder                    13214
A0201_GILGFVFTL_Flu-MP_Influenza_binder             5855
A1101_IVTDFSVIK_EBNA-3B_EBV_binder                  4510
A1101_AVFDRKSDAK_EBNA-3B_EBV_binder                 2842
A0201_GLCTLVAML_BMLF1_EBV_binder                    1014
A0301_RLRAEAQVK_EMNA-3A_EBV_binder                   311
A0201_ELAGIGILTV_MART-1_Cancer_binder                186
A0201_LLDFVRFMGV_EBNA-3B_EBV_binder                  108
A0201_FLYALALLL_LMP2A_EBV_binder                     102
B0801_FLRGRAYGL_EBNA-3A_EBV_binder                    31
A0201_RTLNAWVKV_Gag-protein_HIV_binder                28
A0201_LLFGYPVYV_HTLV-1_binder                         23
A0201_SLFNTVATLY_Gag-protein_HIV_binder               19
A2402_AYAQKIFKI_IE-1_CMV_binder                       19
A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder           18
A0201_KTWGQYWQV_gp100_Cancer_binder                   12
A0201_IMDQVPFSV_gp100_Cancer_bi

In [62]:
test_adata.obs['binding_name'].value_counts()

A0301_KLGGALQAK_IE-1_CMV_binder                    5298
B0801_RAKFKQLL_BZLF1_EBV_binder                    3276
A0201_GILGFVFTL_Flu-MP_Influenza_binder            1538
A1101_IVTDFSVIK_EBNA-3B_EBV_binder                 1150
A1101_AVFDRKSDAK_EBNA-3B_EBV_binder                 677
A0201_GLCTLVAML_BMLF1_EBV_binder                    230
A0301_RLRAEAQVK_EMNA-3A_EBV_binder                   71
A0201_ELAGIGILTV_MART-1_Cancer_binder                55
A0201_FLYALALLL_LMP2A_EBV_binder                     22
A0201_LLDFVRFMGV_EBNA-3B_EBV_binder                  19
A0201_SLFNTVATLY_Gag-protein_HIV_binder               9
A0201_RTLNAWVKV_Gag-protein_HIV_binder                7
A0201_LLFGYPVYV_HTLV-1_binder                         6
A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder           6
A0201_YLLEMLWRL_LMP1_EBV_binder                       5
A2402_AYAQKIFKI_IE-1_CMV_binder                       4
B0801_FLRGRAYGL_EBNA-3A_EBV_binder                    4
A0201_KVLEYVIKV_MAGE-A1_Cancer_binder           

In [75]:
antigen_count = train_adata.obs['binding_name'].value_counts()
high_antigen_count = antigen_count[antigen_count > len(train_adata)*0.01]
high_antigen_count

A0301_KLGGALQAK_IE-1_CMV_binder            21091
B0801_RAKFKQLL_BZLF1_EBV_binder            13214
A0201_GILGFVFTL_Flu-MP_Influenza_binder     5855
A1101_IVTDFSVIK_EBNA-3B_EBV_binder          4510
A1101_AVFDRKSDAK_EBNA-3B_EBV_binder         2842
A0201_GLCTLVAML_BMLF1_EBV_binder            1014
Name: binding_name, dtype: int64

In [84]:
train_adata.obs['binding_name'].isin(high_antigen_count.index)

AAACGGGAGAAGATTC-1-donor_1    True
AACTTTCAGTAACCCT-1-donor_1    True
ACACCCTAGGAGTTTA-1-donor_1    True
ACGAGGAGTCAATGTC-1-donor_1    True
ACGCCAGCAATCAGAA-1-donor_1    True
                              ... 
TGACTTTCAATGAATG-8-donor_4    True
TGGCTGGGTCGACTAT-8-donor_4    True
TGTGTTTGTCCAAGTT-8-donor_4    True
TTCGAAGAGTTATCGC-8-donor_4    True
TTTATGCAGGGCTTCC-8-donor_4    True
Name: binding_name, Length: 49459, dtype: bool

In [85]:
train_adata_reduced = train_adata[train_adata.obs['binding_name'].isin(high_antigen_count.index)]
test_adata_reduced = test_adata[test_adata.obs['binding_name'].isin(high_antigen_count.index)]

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


View of AnnData object with n_obs × n_vars = 48526 × 5000
    obs: 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_cdr3', 'IR_VJ_2_cdr3', 'IR_VDJ_1_cdr3', 'IR_VDJ_2_cdr3', 'IR_VJ_1_cdr3_nt', 'IR_VJ_2_cdr3_nt', 'IR_VDJ_1_cdr3_nt', 'IR_VDJ_2_cdr3_nt', 'IR_VJ_1_expr', 'IR_VJ_2_expr', 'IR_VDJ_1_expr', 'IR_VDJ_2_expr', 'IR_VJ_1_expr_raw', 'IR_VJ_2_expr_raw', 'IR_VDJ_1_expr_raw', 'IR_VDJ_2_expr_raw', 'IR_VJ_1_v_gene', 'IR_VJ_2_v_gene', 'IR_VDJ_1_v_gene', 'IR_VDJ_2_v_gene', 'IR_VJ_1_d_gene', 'IR_VJ_2_d_gene', 'IR_VDJ_1_d_gene', 'IR_VDJ_2_d_gene', 'IR_VJ_1_j_gene', 'IR_VJ_2_j_gene', 'IR_VDJ_1_j_gene', 'IR_VDJ_2_j_gene', 'IR_VJ_1_c_gene', 'IR_VJ_2_c_gene', 'IR_VDJ_1_c_gene', 'IR_VDJ_2_c_gene', 'IR_VJ_1_junction_ins', 'IR_VJ_2_junction_ins', 'IR_VDJ_1_junction_ins', 'IR_VDJ_2_junction_ins', 'has_ir', 'multi_chain', 'barcode', 'donor', 'cell_clono_cdr3_aa', 'cell_clono_cdr3_nt', 'CD3', 'CD19', 'CD45RA', 'CD4', 'CD8a', 'CD14', 'CD45RO', 'CD279_PD-1', 'IgG1', 'IgG2a',

In [63]:
z_train = model.get_latent(
    adatas=[train_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=['binding_name', 'binding_label']
)

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Batch:   0%|                                                                                   | 0/194 [00:00<?, ?it/s]..\tcr_embedding\models\joint_model.py:352: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  z.obs[metadata] = np.array(metadata_batch).T
Batch: 100%|█████████████████████████████████████████████████████████████████████████| 194/194 [00:08<00:00, 23.74it/s]


In [64]:
z_test = model.get_latent(
    adatas=[test_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=['binding_name', 'binding_label']
)

Batch: 100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 23.48it/s]


In [65]:
classes = 'binding_name'
model.kNN(z_train, z_test, classes, 5, 'distance')

C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass weights=distance as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [66]:
from sklearn.metrics import classification_report
print(classification_report(z_test.obs[classes], z_test.obs['pred_'+classes]))
experiment.log_text(text=classification_report(z_test.obs[classes], z_test.obs['pred_'+classes]), step=model.epoch)

COMET WARNING: Empty mapping given to log_params({}); ignoring
C:\Users\yanga\AppData\Local\Continuum\anaconda3\envs\tcr\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                 precision    recall  f1-score   support

A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder       0.00      0.00      0.00         1
      A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder       0.00      0.00      0.00         1
          A0201_ELAGIGILTV_MART-1_Cancer_binder       0.00      0.00      0.00        55
    A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder       0.00      0.00      0.00         6
               A0201_FLYALALLL_LMP2A_EBV_binder       0.95      0.86      0.90        22
        A0201_GILGFVFTL_Flu-MP_Influenza_binder       0.97      0.95      0.96      1538
               A0201_GLCTLVAML_BMLF1_EBV_binder       0.95      0.98      0.96       230
            A0201_IMDQVPFSV_gp100_Cancer_binder       0.00      0.00      0.00         2
            A0201_KTWGQYWQV_gp100_Cancer_binder       0.00      0.00      0.00         2
          A0201_KVLEYVIKV_MAGE-A1_Cancer_binder       0.00      0.00      0.00         3
            A0201_LL

{'web': 'https://www.comet.ml/api/asset/download?assetId=8036eb0d39274eab82be3d846b6f89fe&experimentKey=e64f46cf7b1a44fcaa87b1dac29baf0d',
 'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=8036eb0d39274eab82be3d846b6f89fe&experimentKey=e64f46cf7b1a44fcaa87b1dac29baf0d',
 'assetId': '8036eb0d39274eab82be3d846b6f89fe'}

In [69]:
experiment.log_table('confusion_matrix.csv', pd.crosstab(z_test.obs[classes], z_test.obs['pred_'+classes]))
pd.crosstab(z_test.obs[classes], z_test.obs['pred_'+classes])

pred_binding_name,A0201_ELAGIGILTV_MART-1_Cancer_binder,A0201_FLYALALLL_LMP2A_EBV_binder,A0201_GILGFVFTL_Flu-MP_Influenza_binder,A0201_GLCTLVAML_BMLF1_EBV_binder,A0201_LLDFVRFMGV_EBNA-3B_EBV_binder,A0201_LLFGYPVYV_HTLV-1_binder,A0301_KLGGALQAK_IE-1_CMV_binder,A0301_RLRAEAQVK_EMNA-3A_EBV_binder,A1101_AVFDRKSDAK_EBNA-3B_EBV_binder,A1101_IVTDFSVIK_EBNA-3B_EBV_binder,B0801_FLRGRAYGL_EBNA-3A_EBV_binder,B0801_RAKFKQLL_BZLF1_EBV_binder
binding_name,,,,,,,,,,,,
A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder,0,0,0,0,0,0,1,0,0,0,0,0
A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder,0,0,0,0,0,0,1,0,0,0,0,0
A0201_ELAGIGILTV_MART-1_Cancer_binder,0,0,2,0,0,0,52,0,0,0,0,1
A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder,0,0,1,0,0,0,5,0,0,0,0,0
A0201_FLYALALLL_LMP2A_EBV_binder,0,19,0,0,0,0,3,0,0,0,0,0
A0201_GILGFVFTL_Flu-MP_Influenza_binder,2,0,1464,1,0,0,64,0,0,1,0,6
A0201_GLCTLVAML_BMLF1_EBV_binder,0,0,0,225,0,0,4,0,1,0,0,0
A0201_IMDQVPFSV_gp100_Cancer_binder,0,0,0,0,0,0,2,0,0,0,0,0
A0201_KTWGQYWQV_gp100_Cancer_binder,0,0,0,0,0,0,2,0,0,0,0,0
